In [1]:
with open('input.txt', 'r') as fl:
    patterns = [s.strip() for s in next(fl).split(',')]
    next(fl)
    towels = [ln.strip() for ln in fl]

In [2]:
class PrefixNode:
    def __init__(self, text = ""):
        self.children = {}
        self.text = text
        self.parents = []
        self.n_root_parents = 0
        
    def add_child(self, name, node):
        self.children[name] = node
        node.parents.append(self)
        node.n_root_parents += self.n_root_parents
        return node

In [3]:
print('patterns:',len(patterns),'towels:', len(towels))

patterns: 447 towels: 400


In [4]:
solutions = {}

for towel in towels:

    root = PrefixNode()
    root.n_root_parents = 1
    prefixes = {0:{0:[root]}}
    sub_patterns = [p for p in patterns if p in towel] 

    for i in range(1, len(towel)+1):
        prefix = towel[:i]
        for gram in sub_patterns:
            if prefix.endswith(gram):
                n = len(gram)
                parents = prefixes.get(i-n, {})
                prefix_nodes = prefixes.get(i, {})
                if len(parents) == 0:
                    node = PrefixNode(text = gram)
                    _p = prefix_nodes.get(i-n, []) + [node]
                    prefix_nodes[i-n] = _p
                else:
                    for start, sub_parents in parents.items():
                        node = PrefixNode(text = gram)
                        for parent in sub_parents:
                            node = parent.add_child(gram, node)
                        _p = prefix_nodes.get(start, []) + [node]
                        prefix_nodes[start] = _p
                prefixes[i] = prefix_nodes
                
    solutions[towel] = prefixes

In [5]:
#part 1
print(sum(1 if prefixes.get(len(towel),{}).get(0) is not None else 0
          for towel, prefixes in solutions.items()))

302


In [6]:
#part 2
total = 0
for towel, prefixes in solutions.items():
    for node in prefixes.get(len(towel), {}).get(0, []):
        total += node.n_root_parents
total

771745460576799